In [2]:
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
import re
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader

In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
df = pd.read_csv("/kaggle/input/scdataset/Vunrebility_Dataset.csv")

In [5]:
df.head()

,Unnamed: 0,filename,code,label,label_encoded
0,0,33790.sol,pragma solidity ^0.4.4;\n\ncontract Token {\n\...,./Dataset/reentrancy (RE)/,5
1,1,31454.sol,pragma solidity ^0.4.4;\n\ncontract Token {\n\...,./Dataset/reentrancy (RE)/,5
2,2,40744.sol,contract SendBalance {\n mapping (address => u...,./Dataset/reentrancy (RE)/,5
3,3,39290.sol,/**\n * Originally from https://github.com/Con...,./Dataset/reentrancy (RE)/,5
4,4,39358.sol,pragma solidity ^0.4.4;\n\nlibrary ArrayLib{\n...,./Dataset/reentrancy (RE)/,5


In [6]:
df = df.drop(columns = ['Unnamed: 0', 'filename', 'label'])

In [7]:
df.head()

,code,label_encoded
0,pragma solidity ^0.4.4;\n\ncontract Token {\n\...,5
1,pragma solidity ^0.4.4;\n\ncontract Token {\n\...,5
2,contract SendBalance {\n mapping (address => u...,5
3,/**\n * Originally from https://github.com/Con...,5
4,pragma solidity ^0.4.4;\n\nlibrary ArrayLib{\n...,5


In [8]:
df['label_encoded'].value_counts()

label_encoded
5    1218
7    1199
4     590
0     406
3     366
6     312
1      97
2      97
Name: count, dtype: int64

In [9]:
max_len = 0
for i in df['code']:
    if len(i) > max_len:
        max_len = len(i)
print(max_len)

108950


In [10]:
print(df['code'][0])

pragma solidity ^0.4.4;

contract Token {

    /// @return total amount of tokens
    function totalSupply(uint256) constant returns (uint256 supply) {}

    /// @param _owner The address from which the balance will be retrieved
    /// @return The balance
    function balanceOf(address _owner) constant returns (uint256 balance) {}

    /// @notice send `_value` token to `_to` from `msg.sender`
    /// @param _to The address of the recipient
    /// @param _value The amount of token to be transferred
    /// @return Whether the transfer was successful or not
    function transfer(address _to, uint256 _value) returns (bool success) {}

    /// @notice send `_value` token to `_to` from `_from` on the condition it is approved by `_from`
    /// @param _from The address of the sender
    /// @param _to The address of the recipient
    /// @param _value The amount of token to be transferred
    /// @return Whether the transfer was successful or not
    function transferFrom(address _from, a

#Preprocessing

In [11]:
df_encoded = pd.get_dummies(df, columns = ['label_encoded'], dtype=int)
df_encoded.head()

,code,label_encoded_0,label_encoded_1,label_encoded_2,label_encoded_3,label_encoded_4,label_encoded_5,label_encoded_6,label_encoded_7
0,pragma solidity ^0.4.4;\n\ncontract Token {\n\...,0,0,0,0,0,1,0,0
1,pragma solidity ^0.4.4;\n\ncontract Token {\n\...,0,0,0,0,0,1,0,0
2,contract SendBalance {\n mapping (address => u...,0,0,0,0,0,1,0,0
3,/**\n * Originally from https://github.com/Con...,0,0,0,0,0,1,0,0
4,pragma solidity ^0.4.4;\n\nlibrary ArrayLib{\n...,0,0,0,0,0,1,0,0


In [12]:
solidity_stopwords = [
    "pragma", "interface", "contract", "function", "event", "modifier", "library", "using",
    "string", "uint8", "uint256", "address", "mapping", "bool", "require", "return", "memory",
    "storage", "public", "internal", "view", "returns", "constant", "constructor",
    "_owner", "_balances", "_allowances", "_founder", "_marketing", "_who", "_burntAmount",
    "_from", "_to", "_value", "_timestamp", "_bool", "msg.sender", "totalSupply",
    "balanceOf", "transfer", "allowance", "approve", "transferFrom", "add", "sub", "mul", "div",
    "mod", "changeFounder", "setMinter", "setFurnace", "freezeAccount","solidity","bytes32"
]

def clean_solidity_code(solidity_code):
    # Remove comments (both single-line and multi-line)
    cleaned_code = re.sub(r'//.*?$', '', solidity_code, flags=re.MULTILINE)
    cleaned_code = re.sub(r'/\*.*?\*/', '', cleaned_code, flags=re.DOTALL)

    # Remove special characters and punctuation
    cleaned_code = re.sub(r'[^a-zA-Z0-9\s]', '', cleaned_code)

    # Remove extra whitespace and blank lines, and convert to lowercase
    cleaned_code = '\n'.join(line.strip().lower() for line in cleaned_code.splitlines() if line.strip())

    # Remove common English stop words
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in cleaned_code.split() if word not in stop_words]
    tokens = [token for token in tokens if token not in solidity_stopwords]
    cleaned_code = ' '.join(tokens)

    return cleaned_code

In [13]:
df['code'] = df['code'].apply(clean_solidity_code)

In [14]:
max_len = 0
for i in df['code']:
    if len(i) > max_len:
        max_len = len(i)
print(max_len)

46854


In [15]:
df_0 = df_encoded.drop(columns = ['label_encoded_1', 'label_encoded_2', 'label_encoded_3', 'label_encoded_4', 'label_encoded_5', 'label_encoded_6', 'label_encoded_7'])
df_1 = df_encoded.drop(columns = ['label_encoded_0', 'label_encoded_2', 'label_encoded_3', 'label_encoded_4', 'label_encoded_5', 'label_encoded_6', 'label_encoded_7'])
df_2 = df_encoded.drop(columns = ['label_encoded_0', 'label_encoded_1', 'label_encoded_3', 'label_encoded_4', 'label_encoded_5', 'label_encoded_6', 'label_encoded_7'])
df_3 = df_encoded.drop(columns = ['label_encoded_0', 'label_encoded_1', 'label_encoded_2', 'label_encoded_4', 'label_encoded_5', 'label_encoded_6', 'label_encoded_7'])
df_4 = df_encoded.drop(columns = ['label_encoded_0', 'label_encoded_1', 'label_encoded_2', 'label_encoded_3', 'label_encoded_5', 'label_encoded_6', 'label_encoded_7'])
df_5 = df_encoded.drop(columns = ['label_encoded_0', 'label_encoded_1', 'label_encoded_2', 'label_encoded_3', 'label_encoded_4', 'label_encoded_6', 'label_encoded_7'])
df_6 = df_encoded.drop(columns = ['label_encoded_0', 'label_encoded_1', 'label_encoded_2', 'label_encoded_3', 'label_encoded_4', 'label_encoded_5', 'label_encoded_7'])
df_7 = df_encoded.drop(columns = ['label_encoded_0', 'label_encoded_1', 'label_encoded_2', 'label_encoded_3', 'label_encoded_4', 'label_encoded_5', 'label_encoded_6'])

#Model Building

In [16]:
models =[]

In [17]:
model_name = "bert-base-uncased"

In [18]:
# model_temp = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [19]:
# params = sum(p.numel() for p in model_temp.parameters() if p.requires_grad)
# print(f"The model has {params} trainable parameters.")
tokenizer.model_max_length

512

In [20]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
batch_size = 32
lr = 2e-5
num_epochs = 10
# model_temp.to(device)

In [21]:
def get_dataloader(text, labels):
    input_ids = []
    attention_masks = []
    
    for desc in text:
        encoded_dict = tokenizer.encode_plus(desc, add_special_tokens=True, padding='max_length', truncation=True,
                                                return_attention_mask=True, return_tensors='pt')
        input_ids.append(encoded_dict['input_ids'].to(device))
        attention_masks.append(encoded_dict['attention_mask'].to(device))
    
    input_ids = torch.cat(input_ids, dim=0).to(device)
    attention_mask = torch.cat(attention_masks, dim=0).to(device)
    labels = torch.tensor(labels.to_numpy()).to(device)
    
    dataset = TensorDataset(input_ids, attention_mask, labels)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    return dataloader

In [22]:
def fine_tune_model(model, train_loader, test_loader):
    optim = torch.optim.Adam(model.parameters(), lr = lr)
    criterion = torch.nn.CrossEntropyLoss()

    print("Training Model!!!")

    for epoch in range(num_epochs):
        correct_predictions = 0
        total_predictions = 0
        running_loss = 0.0
        count = 0

        for input_ids, attention_mask, labels in train_loader:
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            labels = labels.to(device)

            optim.zero_grad()

            output = model(input_ids, attention_mask)
            logits = output.logits
            loss = criterion(logits, labels)
            loss.backward()
            optim.step()

            running_loss += loss.item()
            count += 1

            _, predicted = torch.max(logits, 1)
            total_predictions += labels.size(0)
            correct_predictions += (predicted == labels).sum().item()

        total_loss = running_loss / count
        accuracy = correct_predictions / total_predictions * 100
        print(f'Epoch: {epoch + 1}, Training Loss: {running_loss:.4f}, Training Accuracy: {accuracy:.2f}%')

    print("Testing Model!!!")

    with torch.no_grad():
        correct_predictions = 0
        total_predictions = 0
        running_loss = 0.0
        count = 0

        for input_ids, attention_mask, labels in test_loader:
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            labels = labels.to(device)

            output = model(input_ids, attention_mask)
            logits = output.logits
            loss = criterion(logits, labels)

            running_loss += loss.item()
            count += 1

            _, predicted = torch.max(logits, 1)
            total_predictions += labels.size(0)
            correct_predictions += (predicted == labels).sum().item()

        accuracy = correct_predictions / total_predictions * 100
        print(f'Testing Loss: {running_loss:.4f}, Testing Accuracy: {accuracy:.2f}%')

In [23]:
def train_model(curr_df, i):
    x = curr_df['code']
    y = curr_df[f'label_encoded_{i}']
    
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, shuffle=True, random_state=42, stratify=y)
    
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
    if torch.cuda.device_count() > 1:
        print("No. of GPUs: ", torch.cuda.device_count())
        model = torch.nn.DataParallel(model)
    model.to(device)
    
    train_loader = get_dataloader(x_train, y_train)
    test_loader = get_dataloader(x_test, y_test)
    
    fine_tune_model(model, train_loader, test_loader)
    
    models.append(model)

In [24]:
datasets = [df_0, df_1, df_2, df_3, df_4, df_5, df_6, df_7]
for i, dataset in enumerate(datasets):
    print(f'For dataset {i}')
    train_model(dataset, i)

For dataset 0


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


No. of GPUs:  2
Training Model!!!
Epoch: 1, Training Loss: 22.8258, Training Accuracy: 92.50%
Epoch: 2, Training Loss: 14.5483, Training Accuracy: 95.22%
Epoch: 3, Training Loss: 13.5086, Training Accuracy: 95.13%
Epoch: 4, Training Loss: 9.7772, Training Accuracy: 96.85%
Epoch: 5, Training Loss: 8.0920, Training Accuracy: 97.75%
Epoch: 6, Training Loss: 5.8565, Training Accuracy: 98.02%
Epoch: 7, Training Loss: 4.9469, Training Accuracy: 98.60%
Epoch: 8, Training Loss: 3.8342, Training Accuracy: 98.83%
Epoch: 9, Training Loss: 5.8883, Training Accuracy: 98.25%
Epoch: 10, Training Loss: 6.3784, Training Accuracy: 97.78%
Testing Model!!!
Testing Loss: 3.6842, Testing Accuracy: 96.50%
For dataset 1


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

No. of GPUs:  2
Training Model!!!
Epoch: 1, Training Loss: 60.0413, Training Accuracy: 71.85%
Epoch: 2, Training Loss: 52.4845, Training Accuracy: 71.32%
Epoch: 3, Training Loss: 49.8085, Training Accuracy: 73.02%
Epoch: 4, Training Loss: 48.2105, Training Accuracy: 73.05%
Epoch: 5, Training Loss: 46.2056, Training Accuracy: 72.40%
Epoch: 6, Training Loss: 45.5479, Training Accuracy: 73.60%
Epoch: 7, Training Loss: 43.6663, Training Accuracy: 73.37%
Epoch: 8, Training Loss: 43.5776, Training Accuracy: 73.45%
Epoch: 9, Training Loss: 42.6490, Training Accuracy: 74.24%
Epoch: 10, Training Loss: 43.6319, Training Accuracy: 73.42%
Testing Model!!!


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Testing Loss: 11.4554, Testing Accuracy: 73.51%
For dataset 6
No. of GPUs:  2
Training Model!!!
Epoch: 1, Training Loss: 23.9871, Training Accuracy: 92.68%
Epoch: 2, Training Loss: 17.4115, Training Accuracy: 93.64%
Epoch: 3, Training Loss: 12.5262, Training Accuracy: 96.00%
Epoch: 4, Training Loss: 8.7739, Training Accuracy: 97.08%
Epoch: 5, Training Loss: 8.5135, Training Accuracy: 97.55%
Epoch: 6, Training Loss: 6.5651, Training Accuracy: 97.75%
Epoch: 7, Training Loss: 4.8002, Training Accuracy: 98.45%
Epoch: 8, Training Loss: 4.2814, Training Accuracy: 98.69%
Epoch: 9, Training Loss: 3.4242, Training Accuracy: 98.86%
Epoch: 10, Training Loss: 3.1135, Training Accuracy: 98.86%
Testing Model!!!
Testing Loss: 5.7325, Testing Accuracy: 95.68%
For dataset 7


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


No. of GPUs:  2
Training Model!!!
Epoch: 1, Training Loss: 55.0930, Training Accuracy: 71.88%
Epoch: 2, Training Loss: 47.7746, Training Accuracy: 72.26%
Epoch: 3, Training Loss: 45.0720, Training Accuracy: 73.40%
Epoch: 4, Training Loss: 42.9029, Training Accuracy: 73.54%
Epoch: 5, Training Loss: 43.1235, Training Accuracy: 72.75%
Epoch: 6, Training Loss: 42.3170, Training Accuracy: 73.19%
Epoch: 7, Training Loss: 41.7533, Training Accuracy: 73.86%
Epoch: 8, Training Loss: 41.5028, Training Accuracy: 74.74%
Epoch: 9, Training Loss: 41.2904, Training Accuracy: 73.16%
Epoch: 10, Training Loss: 40.5745, Training Accuracy: 74.36%
Testing Model!!!
Testing Loss: 13.8808, Testing Accuracy: 73.86%


In [26]:
for i, model in enumerate(models):
    torch.save(model.state_dict(), f'model_{i}.pth')

In [39]:
# Load the model
new_model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased')

# Load the saved state dict
state_dict = torch.load('/kaggle/working/model_0.pth')

# Remove 'module.' prefix from the state dict keys
new_state_dict = {}
for k, v in state_dict.items():
    new_key = k[7:] if k.startswith('module.') else k  # Remove 'module.' prefix if it exists
    new_state_dict[new_key] = v

# Load the corrected state dict
new_model.load_state_dict(new_state_dict)

new_model = torch.nn.DataParallel(new_model)
new_model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-39-85e226b7fa48>:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We reco

DataParallel(
  (module): BertForSequenceClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSdpaSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=7

In [45]:
def test(model, test_loader):
    criterion = torch.nn.CrossEntropyLoss()
    with torch.no_grad():
        correct_predictions = 0
        total_predictions = 0
        running_loss = 0.0
        count = 0

        for input_ids, attention_mask, labels in test_loader:
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            labels = labels.to(device)

            output = model(input_ids, attention_mask)
            logits = output.logits
            loss = criterion(logits, labels)

            running_loss += loss.item()
            count += 1

            _, predicted = torch.max(logits, 1)
            total_predictions += labels.size(0)
            correct_predictions += (predicted == labels).sum().item()

        accuracy = correct_predictions / total_predictions * 100
        print(f'Testing Loss: {running_loss:.4f}, Testing Accuracy: {accuracy:.2f}%')

In [46]:
x = df_0['code']
y = df_0[f'label_encoded_0']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, shuffle=True, random_state=42, stratify=y)

test_loader = get_dataloader(x_test, y_test)
test(new_model, test_loader)

Testing Loss: 3.6266, Testing Accuracy: 96.50%
